<a href="https://colab.research.google.com/github/chakri9m/Chakri/blob/main/lerobot/training-deffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤗 x 🦾: Training pusht with LeRobot Notebook

Welcome to the **LeRobot pusht training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `pusht` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `diffusion-pusht` policy for 20,000 steps typically takes **about 5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer!

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.


In [6]:
!git clone https://huggingface.co/lerobot/pusht
!cd pusht
!cd lerobot && pip install -e .


Cloning into 'pusht'...
fatal: could not read Username for 'https://huggingface.co': No such device or address
/bin/bash: line 1: cd: pusht: No such file or directory
Obtaining file:///content/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lerobot (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.1.0-py3-none-any.whl size=15888 sha256=88e8cf8847fa9e184c0290072aa1d2ae322e2a7bd3bc941d06e551b3c242541f
  Stored in directory: /tmp/pip-ephem-wheel-cache-460ragwi/wheels/15/0d/02/b9c6ff1c78574dee99101ad231194b3425eb4cd784ce8c8338
Successfully built lerobot
  Attempting uninstall: lerobot
    Found existing installation: lerobot 0.1.0
    Uninstalling lerobot-0.1.0:
      Successfully uninstalled lerobot-0.1.0


In [3]:
!git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

Cloning into 'lerobot'...
remote: Enumerating objects: 28430, done.
remote: Counting objects: 100% (641/641), done.
remote: Compressing objects: 100% (279/279), done.
remote: Total 28430 (delta 471), reused 386 (delta 344), pack-reused 27789 (from 4)
Receiving objects: 100% (28430/28430), 161.51 MiB | 30.75 MiB/s, done.
Resolving deltas: 100% (18215/18215), done.
Filtering content: 100% (45/45), 69.04 MiB | 46.76 MiB/s, done.
Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ffmpeg=7.1.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    alsa-lib-1.2.14            |       hb9d3cd8_0         553 KB  conda-forge
    aom-3.9.1                  |       hac33072_0         2.6 MB  conda-forge
    attr-2.5.1                 |       h166bdaf_1          69 KB  conda-forge

## Weights & Biases login
This cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging.

In [4]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chakravardhan999 (chakravardhan999-sheffield-hallam-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Install pusht dependencies

In [9]:
!pip install -e lerobot


Obtaining file:///content/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lerobot (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.1.0-py3-none-any.whl size=15888 sha256=88e8cf8847fa9e184c0290072aa1d2ae322e2a7bd3bc941d06e551b3c242541f
  Stored in directory: /tmp/pip-ephem-wheel-cache-fxvxz0s9/wheels/15/0d/02/b9c6ff1c78574dee99101ad231194b3425eb4cd784ce8c8338
Successfully built lerobot
  Attempting uninstall: lerobot
    Found existing installation: lerobot 0.1.0
    Uninstalling lerobot-0.1.0:
      Successfully uninstalled lerobot-0.1.0


In [8]:
!cd lerobot && pip install -e


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

-e option requires 1 argument


In [10]:
!ls /content/lerobot


benchmarks	    docker    LICENSE	   media	   src
CODE_OF_CONDUCT.md  docs      Makefile	   pyproject.toml  tests
CONTRIBUTING.md     examples  MANIFEST.in  README.md


In [11]:
!ls /content/lerobot/examples


1_load_lerobot_dataset.py	 4_train_policy_with_script.md	lekiwi
2_evaluate_pretrained_policy.py  advanced
3_train_policy.py		 backward_compatibility


In [12]:
!find /content/lerobot -name train.py


/content/lerobot/src/lerobot/configs/train.py
/content/lerobot/src/lerobot/scripts/train.py


In [17]:
!python /content/lerobot/src/lerobot/scripts/train.py \
    --output_dir=outputs/train/diffusion_pusht \
    --policy.type=diffusion \
    --dataset.repo_id=lerobot/pusht \
    --seed=100000 \
    --env.type=pusht \
    --batch_size=64 \
    --steps=200000 \
    --eval_freq=25000 \
    --save_freq=25000 \
    --wandb.enable=true \
    --policy.repo_id=local



INFO 2025-07-02 12:13:59 ils/utils.py:48 Cuda backend detected, using cuda.
WARNING 2025-07-02 12:13:59 /policies.py:77 Device 'None' is not available. Switching to 'cuda'.
INFO 2025-07-02 12:13:59 ts/train.py:111 {'batch_size': 64,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         'type': 'ColorJitter',
                                                         'weight': 1.0},
                                          'contrast': {'kwargs': {'contrast': [0.8,
                                                                               1.2]},
                                                       'type': 'Colo

In [23]:
!pip install gym-pusht


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 136.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 149.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 104.7 MB/s eta 0:00:00


In [28]:
!python eval.py --help


python: can't open file '/content/lerobot/eval.py': [Errno 2] No such file or directory


In [29]:
!find /content/lerobot -name '*eval*.py'


/content/lerobot/src/lerobot/configs/eval.py
/content/lerobot/src/lerobot/scripts/rl/eval_policy.py
/content/lerobot/src/lerobot/scripts/eval.py
/content/lerobot/examples/2_evaluate_pretrained_policy.py
/content/lerobot/examples/lekiwi/evaluate.py


In [30]:
!python /content/lerobot/src/lerobot/scripts/eval.py --your_arguments_here


usage: eval.py [-h] [--config_path str] [--env str] [--env.obs_type str]
               [--env.render_mode str] [--env.visualization_width str]
               [--env.visualization_height str] [--robot str]
               [--env.robot.type str] [--teleop str] [--env.teleop.type str]
               [--env.task str] [--env.fps str] [--env.features str]
               [--env.features_map str] [--env.type str] [--env.name str]
               [--env.use_viewer str] [--env.gripper_penalty str]
               [--env.use_gamepad str] [--env.state_dim str]
               [--env.action_dim str] [--env.episode_length str]
               [--video_record str] [--env.video_record.enabled str]
               [--env.video_record.record_dir str]
               [--env.video_record.trajectory_name str]
               [--env.reward_classifier_pretrained_path str]
               [--robot_config str] [--env.robot_config.type str]
               [--teleop_config str] [--env.teleop_config.type str]
           

In [26]:
!python /content/lerobot/src/lerobot/scripts/eval.py \
    --policy.path=outputs/train/diffusion_pusht/checkpoints/175000/pretrained_model \
    --output_dir=outputs/eval/diffusion_pusht/175000 \
    --env.type=pusht \
    --eval.n_episodes=500 \
    --eval.batch_size=50 \
    --device=cuda \
    --use_amp=false

usage: eval.py [-h] [--config_path str] [--env str] [--env.obs_type str]
               [--env.render_mode str] [--env.visualization_width str]
               [--env.visualization_height str] [--robot str]
               [--env.robot.type str] [--teleop str] [--env.teleop.type str]
               [--env.task str] [--env.fps str] [--env.features str]
               [--env.features_map str] [--env.type str] [--env.name str]
               [--env.use_viewer str] [--env.gripper_penalty str]
               [--env.use_gamepad str] [--env.state_dim str]
               [--env.action_dim str] [--env.episode_length str]
               [--video_record str] [--env.video_record.enabled str]
               [--env.video_record.record_dir str]
               [--env.video_record.trajectory_name str]
               [--env.reward_classifier_pretrained_path str]
               [--robot_config str] [--env.robot_config.type str]
               [--teleop_config str] [--env.teleop_config.type str]
           

In [20]:
%cd /content/lerobot
!pip install -e .

/content/lerobot
Obtaining file:///content/lerobot
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/pip/_internal/cli/base_command.py", line 105, in _run_wrapper
    status = _inner_run()
             ^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pip/_internal/cli/base_command.py", line 96, in _inner_run
    return self.run(options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pip/_internal/commands/install.py", line 379, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pip/_internal/resolution/resolvelib/resolver.py", line 76, in resolve
    collected = sel

In [18]:
!ls -R /content/lerobot | head -n 100



/content/lerobot:
benchmarks
CODE_OF_CONDUCT.md
CONTRIBUTING.md
docker
docs
examples
LICENSE
Makefile
MANIFEST.in
media
pyproject.toml
README.md
src
tests

/content/lerobot/benchmarks:
video

/content/lerobot/benchmarks/video:
capture_camera_feed.py
README.md
run_video_benchmark.py

/content/lerobot/docker:
lerobot-cpu
lerobot-gpu
lerobot-gpu-dev

/content/lerobot/docker/lerobot-cpu:
Dockerfile

/content/lerobot/docker/lerobot-gpu:
Dockerfile

/content/lerobot/docker/lerobot-gpu-dev:
Dockerfile

/content/lerobot/docs:
README.md
source

/content/lerobot/docs/source:
backwardcomp.mdx
cameras.mdx
contributing.md
hilserl.mdx
hilserl_sim.mdx
il_robots.mdx
il_sim.mdx
index.mdx
installation.mdx
integrate_hardware.mdx
koch.mdx
lekiwi.mdx
notebooks.mdx
smolvla.mdx
so100.mdx
so101.mdx
_toctree.yml

/content/lerobot/examples:
1_load_lerobot_dataset.py
2_evaluate_pretrained_policy.py
3_train_policy.py
4_train_policy_with_script.md
advanced
backward_compatibility
lekiwi

/content/lerobot/examples/a

part 2

In [21]:
!ls /content/lerobot/lerobot/scripts

ls: cannot access '/content/lerobot/lerobot/scripts': No such file or directory


In [ ]:
!python /content/lerobot/lerobot/scripts/eval.py  --help


## Login into Hugging Face Hub
Now after training is done login into the Hugging Face hub and upload the last checkpoint

In [ ]:
!huggingface-cli login

In [ ]:
!huggingface-cli upload ${HF_USER}/my_smolvla \
  /content/lerobot/outputs/train/my_smolvla/checkpoints/last/pretrained_model